## Name: Krina Bharatbhai Patel
## Net Id : vj3004

## Name: Fnu,Sharmila Bolikoppa Palakshappa
## Net Id : bm6575

## Name: Vatsal Sudhir Pathak
## Net Id : pv2556

## Group Id: 7
## Assignment 2

## Q3 [3pts]: Repeat Q2, assuming that all the class covariance matrices are the same. 




In [6]:
import numpy as np
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
import pandas as pd
from numpy.linalg import multi_dot
from sklearn.metrics import confusion_matrix


df_train = read_csv('optdigits_69_N200.tra',header=None)
df_test = read_csv('optdigits_69.tes',header=None)

In [7]:
# computing variance for each columns for each class
# if variance is below or equal to 0(var_threshold) for any column for any class, that column will be dropped

var_threshold = 0
df_var = df_train.groupby(by=64).var()
s_var_threshold = (df_var > var_threshold).product()
columns_to_drop = s_var_threshold[s_var_threshold == 0].index
print(columns_to_drop)

Int64Index([0, 1, 6, 7, 8, 14, 15, 16, 21, 22, 23, 24, 31, 32, 39, 40, 47, 48,
            56],
           dtype='int64')


In [8]:
# drop columns with variance below or equal to  threshold
df_train = df_train.drop(columns = columns_to_drop)
df_test = df_test.drop(columns = columns_to_drop)

## Find statistics (mean, common covariance matrix, common inverse covariance matrix and probability) for each label (6,9)

In [9]:
grouped = df_train.groupby(by=64)
stat_list = []
s = []
common_cov = np.zeros((45,45))

for name, group in grouped:
    df = group.drop(columns=[64])
    m = np.mean(df)
    s = np.cov(df.T)
    common_cov = s + common_cov
    
    s_inv = np.linalg.inv(s)
    prob = df.shape[0] / df_train.shape[0]
    common_covariance = np.dot(common_cov,prob)
    common_covariance_inv = np.linalg.inv(common_covariance)
    stat_list.append([name, m, prob,common_covariance,common_covariance_inv])
    

In [10]:
df_stat = pd.DataFrame(stat_list, columns=['label', 'mean', 'prob', 'common_covariance', 'common_covariance_inv'])
df_stat = df_stat.set_index('label')

## Multivariate discriminant function for common covariance matrix

In [11]:
def discriminant_func(x, mean, common_covariance, common_covariance_inv, prob):   
    
    wi = np.dot(common_covariance_inv,mean)
    wj = np.linalg.multi_dot([mean.T,common_covariance_inv,mean])
    wk = -0.5 * wj + np.log(prob)
    g = np.dot(wi.T, x) + wk 
    return g.squeeze().astype('float')


## Multivariate predict function

In [12]:
def multivariate_predict(df, df_stat):
    df_no_label = df.drop(columns=[64])

    results = []
    for i in range(df_no_label.shape[0]):
        x = np.asarray(df_no_label.iloc[i])
        x = x.reshape(1,-1).T
        discriminant_values = df_stat.apply(lambda row:
                            discriminant_func(x, row["mean"], row["common_covariance"], row["common_covariance_inv"], row["prob"]), axis=1)

        discriminant_values = discriminant_values.astype('float')
        res = discriminant_values.idxmax()
        results.append(res)

    df_results = pd.DataFrame(results, columns=['predicted'])
    return df_results

## Find train and test errors

In [13]:
df_results_train = multivariate_predict(df_train, df_stat)
df_results_test = multivariate_predict(df_test, df_stat)

In [14]:
e = df_results_train.ne(df_train[64], axis=0).mean().sort_values()*100
print("Train error: %.4f %%"%(e.values[0]))

Train error: 11.5000 %


## Train error: 11.5000 %

In [15]:
e = df_results_test.ne(df_test[64], axis=0).mean().sort_values()*100
print("Test error: %.4f %%"%(e.values[0]))

Test error: 12.7424 %


## Test error: 12.7424 %

## Confusion Matrix
## 1) Confusion Matrix for Training

In [16]:
confusion_matrix(df_train[64], df_results_train)

array([[100,   0],
       [ 23,  77]])

## 2) Confusion Matrix for Testing

In [18]:
confusion_matrix(df_test[64], df_results_test)

array([[181,   0],
       [ 46, 134]])